# Lesson 4: Creating an MCP Server

Build a chatbot that makes use of ArXiv for searching for papers and finding some information.

## Import Libraries

In [4]:
! pip install arxiv
! pip install python-dotenv
! pip install anthropic
! pip install fastmcp

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=30ceac4925353fca2a74733afbb6054522f34997a6c1b1d39098f800958603a2
  Stored in directory: /Users/janhavisathe/Library/Caches/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-p

In [2]:
%%writefile mcp_project/research_server.py

import arxiv
import json
import os
from typing import List
import anthropic
from mcp.server.fastmcp import FastMCP

PAPER_DIR = "papers" # define directory name to save papers in

# Initialize FastMCP server
mcp = FastMCP("research")

# Search Papers function
@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
  """
  Search for papers on arXiv based on a topic and store their information.

  Args:
    topic: The topic to search for papers on.
    max_results: The maximum number of results to return. (default: 5)

  Returns:
    List of paper IDs found in the search.
  """

  # Use arxiv to find the papers
  client = arxiv.Client()

  # Search for the most relevant articles matching the queried topic
  search = arxiv.Search(
      query = "all:" + topic,
      max_results = max_results,
      sort_by = arxiv.SortCriterion.Relevance,
  )

  papers = client.results(search)

  # Create directory for this topic
  path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
  os.makedirs(path, exist_ok=True)

  file_path = os.path.join(path, "papers_info.json")

  # Try to load existing papers info
  try:
    with open(file_path, "r") as json_file:
      papers_info = json.load(json_file)
  except (FileNotFoundError, json.JSONDecodeError):
    papers_info = {}

  # Process each paper and add to papers_info
  paper_ids = []
  for paper in papers:
    paper_ids.append(paper.get_short_id())
    paper_info = {
        "title": paper.title,
        "summary": paper.summary,
        "authors": [author.name for author in paper.authors],
        "pdf_url": paper.pdf_url,
        "published": str(paper.published.date())
    }
    papers_info[paper.get_short_id()] = paper_info

  # Save updated papers_info to json file
  with open(file_path, "w") as json_file:
    json.dump(papers_info, json_file, indent=2)

  print(f"Results are saved in: {file_path}")

  return paper_ids



# Extract information from papers
@mcp.tool()
def extract_info(paper_id: str) -> str:
  """
  Search for information about a specific paper across all topic directories

  Args:
    paper_id: The ID of the paper to search for.

  Returns:
    JSON string containing paper information if found, error message if not found.
  """
  for item in os.listdir(PAPER_DIR):
    item_path = os.path.join(PAPER_DIR, item)
    if os.path.isdir(item_path):
      file_path = os.path.join(item_path, "papers_info.json")
      if os.path.isfile(file_path):
        try:
          with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
            if paper_id in papers_info:
              return json.dumps(papers_info[paper_id], indent=2)
        except (FileNotFoundError, json.JSONDecodeError) as e:
          print(f"Error reading {file_path}: {str(e)}")
          continue

  return f"There is no saved information saved about paper {paper_id}."

if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting mcp_project/research_server.py


## Terminal Instructions
- To open the terminal, run the cell below.
- Navigate to the project directory and initiate it with `uv`:
    - `cd L4/mcp_project`
    - `uv init`
-  Create virtual environment and activate it:
    - `uv venv`
    - `source .venv/bin/activate`
- Install dependencies:
    - `uv add mcp arxiv`
- Launch the inspector:
    - `npx @modelcontextprotocol/inspector uv run research_server.py`
    - If you get a message asking "need to install the following packages", type: `y`
- You will get a message saying that the inspector is up and running at a specific address. To open the inspector, click on that given address. The inspector will open in another tab.
- In the inspector UI, make sure to follow the video. You would need to specify under configuration the `Inspector Proxy Address`. Please check the "Inspector UI Instructions" below and run the last cell (after the terminal) to get the `Inspector Proxy Address` for your configurations. 
- If you tested the tool and would like to access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L4` -> `mcp_project`.
- Once you're done with the inspector UI, make sure to close the inspector by typing `Ctrl+C` in the terminal below.

In [3]:
from IPython.display import IFrame
import os

url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/1", width=660, height=600)

AttributeError: 'NoneType' object has no attribute 'format'